In [15]:
%matplotlib notebook
import matplotlib 
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import random

In [59]:
df = pd.read_csv("data/Tongji_375_CN.csv")
df1 = pd.read_csv("data/Tongji_110_CN.csv")
df2 = pd.read_csv("data/St_Antonius_NL.csv")
df3 = pd.read_csv("data/Outcomerea_FR.csv")
df_clustered = pd.read_csv("kmeans_tongji375.csv")
df4 = pd.read_csv("data/Northwell_US.csv")

In [18]:
df4.shape

(300, 11)

In [17]:
df4 = df4.loc[random.sample(range(0,1038), 300),:]

In [19]:
df['type'] = ["Tongji_375"] * df.shape[0]
df1['type'] = ["Tongji_110"] * df1.shape[0]
df2['type'] = ["St_Antonius"] * df2.shape[0]
df3['type'] = ["Outcomerea_FR"] * df3.shape[0]
df4['type'] = ["Northwell_US"] * df4.shape[0]

In [20]:
df_combined = pd.concat([df,df1,df2,df3,df4])

In [18]:
df_combined.head()

,id,admission,discharge,outcome,LDH_first,LDH_last,hsCRP_first,hsCRP_last,lymphocytes_first,lymphocytes_last,type
0,1,2020-01-30 22:12:47,2020-02-17 12:40:09,0,306.0,206.0,43.1,2.6,22.6,25.9,Tongji_375
1,2,2020-02-04 21:39:03,2020-02-19 12:59:01,0,738.0,282.0,27.4,27.4,7.5,9.2,Tongji_375
2,3,2020-01-23 10:59:36,2020-02-08 17:52:31,0,328.0,226.0,42.3,3.6,19.1,27.3,Tongji_375
3,4,2020-01-31 23:03:59,2020-02-18 12:59:12,0,338.0,249.0,108.2,14.5,12.5,16.5,Tongji_375
4,5,2020-02-01 20:59:54,2020-02-18 10:33:06,0,195.0,179.0,7.0,0.8,37.9,23.6,Tongji_375


In [19]:
!pip install plotly

In [25]:
import plotly.express as px

In [21]:
df_combined1 = df_combined.copy()

In [22]:
df_combined1['LDH_last'] = np.log1p(df_combined1['LDH_last'])
df_combined1['hsCRP_last'] = np.log1p(df_combined1['hsCRP_last'])
df_combined1['lymphocytes_last'] = np.log1p(df_combined1['lymphocytes_last'])
df_combined1['died'] = np.where(df_combined1['outcome'] == 1, 'died', 'survived')

In [62]:
fig = px.scatter_3d(df_combined1,
    x = 'LDH_last', 
                   y = 'hsCRP_last', z = 'lymphocytes_last', 
                    color = 'type', 
                    symbol = 'died',
                 title = "Scatterplot of features for each hospital")
fig.show()
fig.write_html("3d_plot_hospitals.html")

In [3]:
df_clustered['LDH_last'] = np.log1p(df_clustered['LDH_last'])
df_clustered['hsCRP_last'] = np.log1p(df_clustered['hsCRP_last'])
df_clustered['lymphocytes_last'] = np.log1p(df_clustered['lymphocytes_last'])

In [61]:
fig1 =  px.scatter_3d(df_clustered, x = 'LDH_last', 
                    y = 'hsCRP_last', z = 'lymphocytes_last', symbol = 'outcome', color = 'cluster',
                     title = "Clusterization results for Tonji hospital")
fig1.show()
fig1.write_html("3d_plot_clusters.html")

In [54]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [37]:
df_combined.columns

Index(['id', 'admission', 'discharge', 'outcome', 'LDH_first', 'LDH_last',
       'hsCRP_first', 'hsCRP_last', 'lymphocytes_first', 'lymphocytes_last',
       'type'],
      dtype='object')

In [39]:
df.head()

,id,admission,discharge,outcome,LDH_first,LDH_last,hsCRP_first,hsCRP_last,lymphocytes_first,lymphocytes_last,type
0,1,2020-01-30 22:12:47,2020-02-17 12:40:09,0,306.0,206.0,43.1,2.6,22.6,25.9,Tongji_375
1,2,2020-02-04 21:39:03,2020-02-19 12:59:01,0,738.0,282.0,27.4,27.4,7.5,9.2,Tongji_375
2,3,2020-01-23 10:59:36,2020-02-08 17:52:31,0,328.0,226.0,42.3,3.6,19.1,27.3,Tongji_375
3,4,2020-01-31 23:03:59,2020-02-18 12:59:12,0,338.0,249.0,108.2,14.5,12.5,16.5,Tongji_375
4,5,2020-02-01 20:59:54,2020-02-18 10:33:06,0,195.0,179.0,7.0,0.8,37.9,23.6,Tongji_375


In [50]:
df5 = df.dropna()

In [63]:
y = np.array(df5['outcome'])
X = df5.drop(['id', 'admission', 'discharge', 'LDH_last', 'hsCRP_last', 'lymphocytes_last', 'type'], axis = 1)

In [64]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)

In [65]:
svm = SVC()

In [66]:
svm.fit(X_train, y_train)

SVC()

In [67]:
 X_train_val=pd.concat((X_train,X_test))
y_train_val=np.concatenate((y_train,y_test), axis=0)

In [68]:
cross_val_score(svm, X_train_val, y_train_val, scoring='accuracy', cv = 20)

array([0.88888889, 0.88888889, 0.77777778, 0.94444444, 0.83333333,
       0.94444444, 0.88888889, 0.88888889, 0.83333333, 0.88888889,
       0.83333333, 0.82352941, 0.88235294, 0.94117647, 0.82352941,
       0.82352941, 0.94117647, 0.94117647, 0.76470588, 0.88235294])